In [1]:

%env GOOGLE_APPLICATION_CREDENTIALS=/Users/klim/Desktop/area51.json

env: GOOGLE_APPLICATION_CREDENTIALS=/Users/klim/Desktop/area51.json


In [2]:
import pandas as pd
from google.cloud import bigquery

client = bigquery.Client()

In [23]:
sql = """
    select distinct(entity) from raw.tweet
    where data_ts > timestamp_sub(current_timestamp(), interval 24 hour)
    order by entity
"""
df = client.query(sql).to_dataframe()
for ent in df.entity: print(ent)

"ada" OR "cardano" (#ada OR #cardano) until:2022-08-07 since:2022-08-06
"ada" OR "cardano" (#ada OR #cardano) until:2022-08-08 since:2022-08-07
"ark" OR "ark invest" (#ark OR #arkinvest) until:2022-08-07 since:2022-08-06
"ark" OR "ark invest" (#ark OR #arkinvest) until:2022-08-08 since:2022-08-07
"avax" OR "avalanche" (#avax OR #avalanche) until:2022-08-07 since:2022-08-06
"avax" OR "avalanche" (#avax OR #avalanche) until:2022-08-08 since:2022-08-07
"binance" OR "binance.us" (#binance) until:2022-08-07 since:2022-08-06
"binance" OR "binance.us" (#binance) until:2022-08-08 since:2022-08-07
"blockchain" (#blockchain) until:2022-08-07 since:2022-08-06
"bnb" OR "binance coin" OR "bsc" OR "binance smart chain" (#bnb OR #bsc OR #binancecoin OR #binancesmartchain) until:2022-08-07 since:2022-08-06
"bnb" OR "binance coin" OR "bsc" OR "binance smart chain" (#bnb OR #bsc OR #binancecoin OR #binancesmartchain) until:2022-08-08 since:2022-08-07
"btc" OR "bitcoin" (#btc OR #bitcoin) until:2022-08-0

In [ ]:
sql = """
    update raw.tweet
    set entity = '"dao" (#dao) until:2022-08-08 since:2022-08-07'
    where
        entity = '"dao" ("#dao") until:2022-08-08 since:2022-08-07'
        and data_ts < timestamp_sub(current_timestamp(), interval 30 minute)
"""
print(sql)
df = client.query(sql).to_dataframe()
df

In [ ]:
sql = """
    delete from raw.tweet
    where 
        entity like '"crypto" OR "cryptocurrency" (#crypto OR #cryptocurrency) until:2022-08-08 since:2022-08-07"blockchain" (#blockchain)%'
        and data_ts < timestamp_sub(current_timestamp(), interval 40 minute)
"""
print(sql)
df = client.query(sql).to_dataframe()
df

In [41]:
sql = """
with tweets as (

    select 
    
        id
        , url
        , created_timestamp

    from `area51-lab`.`dim`.`vitalik_buterin_tweets`

)

, mentioned_tokens as (

    select 
    
        id
        , url
        , token

    from 
        `area51-lab`.`dim`.`tweet_mentioned_tokens`
        , unnest(tokens) token

)
    
, joined as (
    select
        extract(week from a.created_timestamp) week
        , token
    from
        tweets a
    left join
        mentioned_tokens b
    on
        a.id = b.id
        and a.url = b.url
)

select * from joined 
pivot(
    count(*) for token in 
    ("ada", "bnb", "btc", "eth", "erg", "matic", "sol", "avax", "doge", "shib", "link", "xrp")
)

"""
print(sql)
df = client.query(sql).to_dataframe()
df


with tweets as (

    select 
    
        id
        , url
        , created_timestamp

    from `area51-lab`.`dim`.`vitalik_buterin_tweets`

)

, mentioned_tokens as (

    select 
    
        id
        , url
        , token

    from 
        `area51-lab`.`dim`.`tweet_mentioned_tokens`
        , unnest(tokens) token

)
    
, joined as (
    select
        extract(week from a.created_timestamp) week
        , token
    from
        tweets a
    left join
        mentioned_tokens b
    on
        a.id = b.id
        and a.url = b.url
)

select * from joined 
pivot(
    count(*) for token in 
    ("ada", "bnb", "btc", "eth", "erg", "matic", "sol", "avax", "doge", "shib", "link", "xrp")
)




,week,ada,bnb,btc,eth,erg,matic,sol,avax,doge,shib,link,xrp
0,30,21,5,52,209,62,1,11,0,1,4,8,2
1,31,93,42,573,1565,164,30,69,8,33,63,16,30
2,27,24,0,36,30,2,0,0,0,0,0,1,0
3,32,1,0,4,6,2,0,0,0,1,0,0,0
4,29,2,1,2,4,0,0,0,0,0,0,0,0
5,18,0,0,1,1,0,0,0,0,0,0,0,0
6,26,0,0,0,1,0,0,0,0,0,0,0,0
7,28,1,0,2,1,1,0,0,0,0,0,0,0


In [29]:
pivot(
    count(*) for token in 
    ("ada", "bnb", "btc", "eth", "erg", "matic", "sol", "avax", "doge", "shib", "link", "xrp")
)

,id,url,token
0,1555671559540465664,https://twitter.com/AgAnttoken/status/15556715...,bnb
1,1555671559540465664,https://twitter.com/AgAnttoken/status/15556715...,btc
2,1555671559540465664,https://twitter.com/AgAnttoken/status/15556715...,eth


In [28]:
sql = """
select 
    id, 
    content,
    if ( regexp_contains(lower(content), r"(bnb|binance coin|binance smart chain|bsc)"), "bnb", null)
from 
    fct.tweets
where
    id = '1555671559540465664'
"""
df = client.query(sql).to_dataframe()
df.head()

,id,content,f0_
0,1555671559540465664,➡️Silver price per Gram $0.64\n➡️$AgAnt tracks...,bnb


In [24]:
df.content[0]

'➡️Silver price per Gram $0.64\n➡️$AgAnt tracks the price of silver per gram.\n➡️AgAnt Contract 0x0c920ac31aF492674fdccb36D7aAA3e373f7c0C3\n\n#AgAntWorld #AgAnt #Crypto #btc #bnb #eth #Silver #gold #cro #bsc #vitalik #memes #justinemusk #Ryoshi #altcoin #news #pancakeswap #semperfi https://t.co/TOAVXyrmpn'